In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import holidays

In [2]:
german_provinces = holidays.DE.PROVINCES

In [3]:
holiday_dict = {}
for province in german_provinces:
    holiday_dict[province] = holidays.DE(years=[2019,2020,2021],prov=province)

In [4]:
wtr = pd.read_csv('weather_data.csv')
# wtr = wtr[wtr.district_name != 'LK Göttingen (alt)']

In [5]:
wtr['date'] = pd.to_datetime(wtr['date'])

In [6]:
wtr['day'] = wtr.date.dt.weekday
wtr['week_no'] = wtr.date.dt.week

In [7]:
# wtr.drop_duplicates(inplace=True)

In [8]:
prov_ags_dict = {8:'BW',
                 9:'BY',
                 11:'BE',
                 12:'BB',
                 4:'HB',
                 2:'HH',
                 6:'HE',
                 13:'MV',
                 3:'NI',
                 5:'NW',
                 7:'RP',
                 10:'SL',
                 14:'SN',
                 15:'ST',
                 1:'SH',
                 16:'TH'}

wtr['province'] = np.floor(wtr['districtId'] / 1000).map(prov_ags_dict)

In [9]:
wtr['holiday'] = None
for i in wtr.index:
    wtr.holiday.at[i] = wtr.date[i] in holiday_dict[wtr.province[i]]

wtr['weekend'] = wtr['day'].isin([5,6])

wtr['day_off'] = wtr['holiday'] | wtr['weekend']

wtr['workday'] = ~wtr['day_off']

In [10]:
# for wcol in ['temperature','humidity','precipitation','sunshine','velocity','direction']:
#     for work in ['workday','day_off']:
#         var_name = wcol + '_' + work
#         wtr[var_name] = wtr[wcol] * wtr[work]

In [11]:
wtr_work = wtr[wtr.workday].copy()
wtr_off = wtr[wtr.day_off].copy()

In [12]:
wtr_cols = ['temperature','humidity','precipitation','sunshine','velocity','direction']
out_dfs = []
for df in [wtr_work,wtr_off]:
    df_ = df.groupby(['districtId','week_no'])[wtr_cols].mean().reset_index()
    out_dfs.append(df_)

In [13]:
wtr_fts = pd.merge(out_dfs[0],out_dfs[1],on=['districtId','week_no'],suffixes=['_workday','_day_off'])

In [14]:
wtr_fts.to_csv('weather_features.csv',index=False)